In [54]:
%load_ext autoreload
%autoreload 2

from typing import List, Dict
from google.transit import gtfs_realtime_pb2 as gtfs_rt
from protobuf_to_dict import protobuf_to_dict
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import re
from datetime import datetime
import sys, os

from pandas.io.json import json_normalize
from itertools import groupby

# TODO better way to import module?
sys.path.insert(0, os.path.realpath('/content/jupyter/mta-accessibility/notebooks/routing'))
import GTFS_Utils as gu

pd.set_option('max_colwidth', 200)

In [56]:
def parse_trip_updates(realtime_data: List[Dict]) -> pd.DataFrame:
    # Get rid of any vehicle update rows
    realtime_data = [d for d in realtime_data if 'vehicle' not in d]    
    df = json_normalize(realtime_data)
    
    # Get rid of null stop_time_updates
    df = df[~df['trip_update.stop_time_update'].isnull()]

    idx = df.index.repeat(df['trip_update.stop_time_update'].str.len())
    df1 = pd.DataFrame({'trip_update.stop_time_update': np.concatenate(df['trip_update.stop_time_update'].values)})
    df1.index = idx
    df = df1.join(df.drop('trip_update.stop_time_update', axis=1), how='left')
    df = df.reset_index(drop=True)
    d2 = json_normalize(df['trip_update.stop_time_update'])
    df = pd.concat([d2, df.drop('trip_update.stop_time_update', axis=1)], axis=1)
    df = df.drop(list(df.filter(regex='vehicle')), axis=1) 
    return df

In [67]:
file_names = gu.get_file_names('/content/jupyter/20190601', 'ace', '20190601')
realtime_data = gu.load_gtfs(file_names[:10])
df = parse_trip_updates(realtime_data)
df['arrival.time'] = pd.to_datetime(df['arrival.time'], unit='s')

df2 = df[df['trip_update.trip.trip_id'] == '038900_E..S']
df2.drop_duplicates()

,arrival.time,departure.time,stop_id,id,trip_update.trip.route_id,trip_update.trip.start_date,trip_update.trip.start_time,trip_update.trip.trip_id
10321,2019-06-01 10:29:00,1559384940,G05S,000017E,E,20190601,06:29:00,038900_E..S
10322,2019-06-01 10:30:30,1559385030,G06S,000017E,E,20190601,06:29:00,038900_E..S
10323,2019-06-01 10:33:00,1559385180,G07S,000017E,E,20190601,06:29:00,038900_E..S
10324,2019-06-01 10:34:30,1559385270,F05S,000017E,E,20190601,06:29:00,038900_E..S
10325,2019-06-01 10:36:30,1559385390,F06S,000017E,E,20190601,06:29:00,038900_E..S
10326,2019-06-01 10:38:00,1559385480,F07S,000017E,E,20190601,06:29:00,038900_E..S
10327,2019-06-01 10:40:00,1559385600,G08S,000017E,E,20190601,06:29:00,038900_E..S
10328,2019-06-01 10:47:00,1559386020,G14S,000017E,E,20190601,06:29:00,038900_E..S
10329,2019-06-01 10:53:00,1559386380,G21S,000017E,E,20190601,06:29:00,038900_E..S
10330,2019-06-01 10:54:30,1559386470,F09S,000017E,E,20190601,06:29:00,038900_E..S


In [44]:
# file_names = get_file_names('20190601', 'j', '20190601')
# file_names.sort()
# file_names

file_names = ['20190601/gtfs_j_20190601_073344.gtfs',
 '20190601/gtfs_j_20190601_073355.gtfs',
 '20190601/gtfs_j_20190601_073414.gtfs',
 '20190601/gtfs_j_20190601_073426.gtfs',
 '20190601/gtfs_j_20190601_073444.gtfs',
 '20190601/gtfs_j_20190601_073456.gtfs',
 '20190601/gtfs_j_20190601_073516.gtfs',
 '20190601/gtfs_j_20190601_073526.gtfs',]



In [47]:
file_names = ['20190601/gtfs_j_20190601_073355.gtfs']
realtime = load_gtfs(file_names)
df = parse_data(realtime)
df['arrival.time'] = pd.to_datetime(df['arrival.time'], unit='s').dt.tz_localize('UTC').dt.tz_convert('America/New_York')

df2 = df[df['trip_update.trip.trip_id'] == '043205_J..S']
df2.sort_values(['arrival.time', 'trip_update.trip.trip_id', 'id'])

,arrival.time,departure.time,stop_id,id,trip_update.trip.route_id,trip_update.trip.start_date,trip_update.trip.start_time,trip_update.trip.trip_id
2,2019-06-01 07:34:55-04:00,1559388895,M20S,000003J,J,20190601,07:12:03,043205_J..S
3,2019-06-01 07:36:25-04:00,1559388985,M21S,000003J,J,20190601,07:12:03,043205_J..S
4,2019-06-01 07:37:55-04:00,1559389075,M22S,000003J,J,20190601,07:12:03,043205_J..S
5,2019-06-01 07:39:25-04:00,1559389165,M23S,000003J,J,20190601,07:12:03,043205_J..S


In [49]:
file_names = ['20190601/gtfs_j_20190601_073414.gtfs']
realtime = load_gtfs(file_names)
df = parse_data(realtime)
df['arrival.time'] = pd.to_datetime(df['arrival.time'], unit='s').dt.tz_localize('UTC').dt.tz_convert('America/New_York')

df2 = df[df['trip_update.trip.trip_id'] == '043205_J..S']
df2.sort_values(['arrival.time', 'trip_update.trip.trip_id', 'id'])

,arrival.time,departure.time,stop_id,id,trip_update.trip.route_id,trip_update.trip.start_date,trip_update.trip.start_time,trip_update.trip.trip_id
8,2019-06-01 07:34:04-04:00,1559388844,M20S,000005J,J,20190601,07:12:03,043205_J..S
9,2019-06-01 07:35:25-04:00,1559388925,M21S,000005J,J,20190601,07:12:03,043205_J..S
10,2019-06-01 07:36:55-04:00,1559389015,M22S,000005J,J,20190601,07:12:03,043205_J..S
11,2019-06-01 07:38:25-04:00,1559389105,M23S,000005J,J,20190601,07:12:03,043205_J..S


In [53]:
file_names = ['20190601/gtfs_j_20190601_073344.gtfs',
 '20190601/gtfs_j_20190601_073355.gtfs',
 '20190601/gtfs_j_20190601_073414.gtfs',
 '20190601/gtfs_j_20190601_073426.gtfs',
 '20190601/gtfs_j_20190601_073444.gtfs',
 '20190601/gtfs_j_20190601_073456.gtfs',
 '20190601/gtfs_j_20190601_073516.gtfs',
 '20190601/gtfs_j_20190601_073526.gtfs',]

day_df = pd.DataFrame()

for f in file_names:        
    realtime = load_gtfs([f])
    df = parse_data(realtime)
    df['file'] = f
    day_df = day_df.append(df)
    
day_df['arrival.time'] = pd.to_datetime(day_df['arrival.time'], unit='s').dt.tz_localize('UTC').dt.tz_convert('America/New_York')

df2 = day_df[day_df['trip_update.trip.trip_id'] == '043205_J..S']
df2.sort_values(['arrival.time', 'trip_update.trip.trip_id', 'id'])

,arrival.time,departure.time,stop_id,id,trip_update.trip.route_id,trip_update.trip.start_date,trip_update.trip.start_time,trip_update.trip.trip_id,file
8,2019-06-01 07:34:04-04:00,1559388844,M20S,000005J,J,20190601,07:12:03,043205_J..S,20190601/gtfs_j_20190601_073414.gtfs
6,2019-06-01 07:34:14-04:00,1559388854,M20S,000005J,J,20190601,07:12:03,043205_J..S,20190601/gtfs_j_20190601_073426.gtfs
6,2019-06-01 07:34:39-04:00,1559388879,M20S,000003J,J,20190601,07:12:03,043205_J..S,20190601/gtfs_j_20190601_073444.gtfs
9,2019-06-01 07:34:43-04:00,1559388883,M20S,000005J,J,20190601,07:12:03,043205_J..S,20190601/gtfs_j_20190601_073344.gtfs
0,2019-06-01 07:34:49-04:00,1559388889,M20S,000001J,J,20190601,07:12:03,043205_J..S,20190601/gtfs_j_20190601_073456.gtfs
2,2019-06-01 07:34:55-04:00,1559388895,M20S,000003J,J,20190601,07:12:03,043205_J..S,20190601/gtfs_j_20190601_073355.gtfs
0,2019-06-01 07:34:59-04:00,1559388899,M20S,000001J,J,20190601,07:12:03,043205_J..S,20190601/gtfs_j_20190601_073516.gtfs
6,2019-06-01 07:34:59-04:00,1559388899,M20S,000003J,J,20190601,07:12:03,043205_J..S,20190601/gtfs_j_20190601_073526.gtfs
9,2019-06-01 07:35:25-04:00,1559388925,M21S,000005J,J,20190601,07:12:03,043205_J..S,20190601/gtfs_j_20190601_073414.gtfs
7,2019-06-01 07:35:25-04:00,1559388925,M21S,000005J,J,20190601,07:12:03,043205_J..S,20190601/gtfs_j_20190601_073426.gtfs


In [9]:
df[df.stop_name.str.contains('Canal')]

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station
96,135,NaN,Canal St,NaN,40.722854,-74.006277,NaN,NaN,1,NaN
97,135N,NaN,Canal St,NaN,40.722854,-74.006277,NaN,NaN,0,135
98,135S,NaN,Canal St,NaN,40.722854,-74.006277,NaN,NaN,0,135
459,639,NaN,Canal St,NaN,40.718803,-74.000193,NaN,NaN,1,NaN
460,639N,NaN,Canal St,NaN,40.718803,-74.000193,NaN,NaN,0,639
461,639S,NaN,Canal St,NaN,40.718803,-74.000193,NaN,NaN,0,639
615,A34,NaN,Canal St,NaN,40.720824,-74.005229,NaN,NaN,1,NaN
616,A34N,NaN,Canal St,NaN,40.720824,-74.005229,NaN,NaN,0,A34
617,A34S,NaN,Canal St,NaN,40.720824,-74.005229,NaN,NaN,0,A34
1254,M20,NaN,Canal St,NaN,40.718092,-73.999892,NaN,NaN,1,NaN


In [3]:
import google.protobuf.json_format
import json
from pandas.io.json import json_normalize

file_name = '20190601/gtfs_nqrw_20190601_235952.gtfs'
file_name = '20190601/gtfs_ace_20190601_072836.gtfs'
feed = load_single_gtfs(file_name)

# The MTA data feed uses the General Transit Feed Specification (GTFS) which
# is based upon Google's "protocol buffer" data format. While possible to
# manipulate this data natively in python, it is far easier to use the
# "pip install --upgrade gtfs-realtime-bindings" library which can be found on pypi
subway_feed = protobuf_to_dict(feed) # subway_feed is a dictionary
realtime_data = subway_feed['entity'] # train_data is a list
realtime_data = [d for d in realtime_data if 'vehicle' not in d]


realtime_data[:2]



[{'id': '000001A',
  'trip_update': {'trip': {'trip_id': '035551_A..S',
    'start_time': '05:55:31',
    'start_date': '20190601',
    'route_id': 'A'},
   'stop_time_update': [{'arrival': {'time': 1559388609},
     'departure': {'time': 1559388609},
     'stop_id': 'A40S'},
    {'arrival': {'time': 1559388759},
     'departure': {'time': 1559388759},
     'stop_id': 'A41S'},
    {'arrival': {'time': 1559388879},
     'departure': {'time': 1559388879},
     'stop_id': 'A42S'},
    {'arrival': {'time': 1559389179},
     'departure': {'time': 1559389179},
     'stop_id': 'A46S'},
    {'arrival': {'time': 1559389329},
     'departure': {'time': 1559389329},
     'stop_id': 'A48S'},
    {'arrival': {'time': 1559389539},
     'departure': {'time': 1559389539},
     'stop_id': 'A51S'},
    {'arrival': {'time': 1559388504},
     'departure': {'time': 1559388504},
     'stop_id': 'H04S'},
    {'arrival': {'time': 1559388571},
     'departure': {'time': 1559388571},
     'stop_id': 'H06S'},
  

In [381]:
def parse_data2(realtime_data):
    df = json_normalize(realtime_data)

    # Get rid of null stop_time_updates
    df = df[~df['trip_update.stop_time_update'].isnull()]

    idx = df.index.repeat(df['trip_update.stop_time_update'].str.len())
    df1 = pd.DataFrame({'trip_update.stop_time_update': np.concatenate(df['trip_update.stop_time_update'].values)})
    df1.index = idx
    df = df1.join(df.drop('trip_update.stop_time_update', axis=1), how='left')
    df = df.reset_index(drop=True)
    d2 = json_normalize(df['trip_update.stop_time_update'])
    return pd.concat([d2, df.drop('trip_update.stop_time_update', axis=1)], axis=1)

In [368]:
def parse_data(realtime_data):
    df = pd.DataFrame.from_dict(realtime_data)
    
    # Horizontally explode trip_update dict
    df = df['trip_update'].apply(pd.Series)
    # Horizontally explode trip_id dict
    df = pd.concat([df.trip.apply(pd.Series), df.stop_time_update], axis=1)
    # Get rid of null stop_time_updates
    df = df[~df.stop_time_update.isnull()]

    # Explode out each row of stop_time_update
    idx = df.index.repeat(df['stop_time_update'].str.len())
    df1 = pd.DataFrame({'stop_time_update': np.concatenate(df.stop_time_update.values)})
    df1.index = idx
    df = df1.join(df.drop('stop_time_update', axis=1), how='left')

    # Explode horizontally arrival,departure, and stop_id
    df1 = df.stop_time_update.apply(pd.Series)
    df = pd.concat([df.drop(['stop_time_update'], axis=1), df1], axis=1)

    # Convert arrival and departure times to datetime.
    df.arrival = df.arrival.apply(pd.Series)
    df.arrival = pd.to_datetime(df.arrival, unit='s')

    df.departure = df.departure.apply(pd.Series)
    df.departure = pd.to_datetime(df.departure, unit='s')
    return df

In [386]:
df = json_normalize(realtime_data)

# Get rid of null stop_time_updates
df = df[~df['trip_update.stop_time_update'].isnull()]

idx = df.index.repeat(df['trip_update.stop_time_update'].str.len())
df1 = pd.DataFrame({'trip_update.stop_time_update': np.concatenate(df['trip_update.stop_time_update'].values)})
df1.index = idx
df = df1.join(df.drop('trip_update.stop_time_update', axis=1), how='left')
df = df.reset_index(drop=True)
df.head()

,trip_update.stop_time_update,id,trip_update.trip.route_id,trip_update.trip.start_date,trip_update.trip.start_time,trip_update.trip.trip_id
0,"{'arrival': {'time': 1559388609}, 'departure': {'time': 1559388609}, 'stop_id': 'A40S'}",000001A,A,20190601,05:55:31,035551_A..S
1,"{'arrival': {'time': 1559388759}, 'departure': {'time': 1559388759}, 'stop_id': 'A41S'}",000001A,A,20190601,05:55:31,035551_A..S
2,"{'arrival': {'time': 1559388879}, 'departure': {'time': 1559388879}, 'stop_id': 'A42S'}",000001A,A,20190601,05:55:31,035551_A..S
3,"{'arrival': {'time': 1559389179}, 'departure': {'time': 1559389179}, 'stop_id': 'A46S'}",000001A,A,20190601,05:55:31,035551_A..S
4,"{'arrival': {'time': 1559389329}, 'departure': {'time': 1559389329}, 'stop_id': 'A48S'}",000001A,A,20190601,05:55:31,035551_A..S


In [387]:



d2 = json_normalize(df['trip_update.stop_time_update'])
pd.concat([d2, df.drop('trip_update.stop_time_update', axis=1)], axis=1).head()

,arrival.time,departure.time,stop_id,id,trip_update.trip.route_id,trip_update.trip.start_date,trip_update.trip.start_time,trip_update.trip.trip_id
0,1559388609,1559388609,A40S,000001A,A,20190601,05:55:31,035551_A..S
1,1559388759,1559388759,A41S,000001A,A,20190601,05:55:31,035551_A..S
2,1559388879,1559388879,A42S,000001A,A,20190601,05:55:31,035551_A..S
3,1559389179,1559389179,A46S,000001A,A,20190601,05:55:31,035551_A..S
4,1559389329,1559389329,A48S,000001A,A,20190601,05:55:31,035551_A..S


In [370]:
%%time 
parse_data(realtime_data).head()

CPU times: user 3.59 s, sys: 0 ns, total: 3.59 s
Wall time: 3.67 s


,trip_id,start_time,start_date,route_id,arrival,departure,stop_id
0,035551_A..S,05:55:31,20190601,A,2019-06-01 11:30:09,2019-06-01 11:30:09,A40S
0,035551_A..S,05:55:31,20190601,A,2019-06-01 11:32:39,2019-06-01 11:32:39,A41S
0,035551_A..S,05:55:31,20190601,A,2019-06-01 11:34:39,2019-06-01 11:34:39,A42S
0,035551_A..S,05:55:31,20190601,A,2019-06-01 11:39:39,2019-06-01 11:39:39,A46S
0,035551_A..S,05:55:31,20190601,A,2019-06-01 11:42:09,2019-06-01 11:42:09,A48S


In [293]:
row = rd[0]
row.keys()

dict_keys(['id', 'trip_update'])

In [12]:
%%time
# TODO API to load data for selected days or day range, selected trains

df = pd.read_pickle('hourly/20190601_07.pkl')
df.columns = df.columns.str.replace('trip_update.trip.', '')
df.columns = df.columns.str.replace('.', '_')

df.arrival_time = pd.to_datetime(df.arrival_time, unit='s').dt.tz_localize('UTC').dt.tz_convert('America/New_York')
#df.departure_time = pd.to_datetime(df.departure_time, unit='s').dt.tz_localize('UTC').dt.tz_convert('America/New_York')

# Join with static GTFS data.
stop_df = pd.read_csv('google_transit/stops.txt')
df = df.merge(stop_df[['stop_id', 'stop_name']], on='stop_id')

df.head()

CPU times: user 24.9 s, sys: 1.11 s, total: 26 s
Wall time: 26.6 s


,arrival_time,departure_time,schedule_relationship,stop_id,id,route_id,start_date,start_time,trip_id,stop_name
0,2019-06-01 07:54:25-04:00,1.559390e+09,0.0,L28S,1,L,20190601,NaN,043550_L..S,E 105 St
1,2019-06-01 08:16:18-04:00,1.559391e+09,0.0,L28S,7,L,20190601,NaN,045550_L..S,E 105 St
2,2019-06-01 08:12:17-04:00,1.559391e+09,0.0,L28S,11,L,20190601,NaN,046350_L..S,E 105 St
3,2019-06-01 08:40:45-04:00,1.559393e+09,0.0,L28S,15,L,20190601,NaN,047550_L..S,E 105 St
4,2019-06-01 08:30:45-04:00,1.559392e+09,0.0,L28S,19,L,20190601,NaN,048350_L..S,E 105 St


In [292]:
row

{'id': '000001A',
 'trip_update': {'trip': {'trip_id': '035551_A..S',
   'start_time': '05:55:31',
   'start_date': '20190601',
   'route_id': 'A'},
  'stop_time_update': [{'arrival': {'time': 1559388609},
    'departure': {'time': 1559388609},
    'stop_id': 'A40S'},
   {'arrival': {'time': 1559388759},
    'departure': {'time': 1559388759},
    'stop_id': 'A41S'},
   {'arrival': {'time': 1559388879},
    'departure': {'time': 1559388879},
    'stop_id': 'A42S'},
   {'arrival': {'time': 1559389179},
    'departure': {'time': 1559389179},
    'stop_id': 'A46S'},
   {'arrival': {'time': 1559389329},
    'departure': {'time': 1559389329},
    'stop_id': 'A48S'},
   {'arrival': {'time': 1559389539},
    'departure': {'time': 1559389539},
    'stop_id': 'A51S'},
   {'arrival': {'time': 1559388504},
    'departure': {'time': 1559388504},
    'stop_id': 'H04S'},
   {'arrival': {'time': 1559388571},
    'departure': {'time': 1559388571},
    'stop_id': 'H06S'},
   {'arrival': {'time': 1559388

In [269]:
df = pd.DataFrame.from_dict(realtime_data)
# Dump all vehicle updates
df = df[df.vehicle.isnull()].drop(columns=['vehicle'])
# Horizontally explode trip_update dict
df = df['trip_update'].apply(pd.Series)
# Horizontally explode trip_id dict
df = pd.concat([df.trip.apply(pd.Series), df.stop_time_update], axis=1)
# Get rid of null stop_time_updates
df = df[~df.stop_time_update.isnull()]

# Explode out each row of stop_time_update
idx = df.index.repeat(df['stop_time_update'].str.len())
df1 = pd.DataFrame({'stop_time_update': np.concatenate(df.stop_time_update.values)})
df1.index = idx
df = df1.join(df.drop('stop_time_update', axis=1), how='left')

# Explode horizontally arrival,departure, and stop_id
df1 = df.stop_time_update.apply(pd.Series)
df = pd.concat([df.drop(['stop_time_update'], axis=1), df1], axis=1)

# Convert arrival and departure times to datetime.
df.arrival = df.arrival.apply(pd.Series)
df.arrival = pd.to_datetime(df.arrival, unit='s')

df.departure = df.departure.apply(pd.Series)
df.departure = pd.to_datetime(df.departure, unit='s')

df.head()

,trip_id,start_time,start_date,route_id,arrival,departure,stop_id
0,035551_A..S,05:55:31,20190601,A,2019-06-01 11:30:09,2019-06-01 11:30:09,A40S
0,035551_A..S,05:55:31,20190601,A,2019-06-01 11:32:39,2019-06-01 11:32:39,A41S
0,035551_A..S,05:55:31,20190601,A,2019-06-01 11:34:39,2019-06-01 11:34:39,A42S
0,035551_A..S,05:55:31,20190601,A,2019-06-01 11:39:39,2019-06-01 11:39:39,A46S
0,035551_A..S,05:55:31,20190601,A,2019-06-01 11:42:09,2019-06-01 11:42:09,A48S


In [270]:
len(df.index)

1768

## Join with static GTFS data

In [261]:
stop_df = pd.read_csv('google_transit/stops.txt')
df = df.merge(stop_df[['stop_id', 'stop_name']], on='stop_id')


In [264]:
df[df.stop_name.str.contains('Inwood')]

,trip_id,start_time,start_date,route_id,arrival,departure,stop_id,stop_name
338,037150_A..N,06:11:30,20190601,A,2019-06-01 11:52:31,2019-06-01 11:52:31,A02N,Inwood - 207 St
339,038200_A..N,06:22:00,20190601,A,2019-06-01 11:35:57,2019-06-01 11:35:57,A02N,Inwood - 207 St
340,039163_A..N,06:31:38,20190601,A,2019-06-01 12:11:54,2019-06-01 12:11:54,A02N,Inwood - 207 St
341,040250_A..N,06:42:30,20190601,A,2019-06-01 11:59:19,2019-06-01 11:59:19,A02N,Inwood - 207 St
342,041150_A..N,06:51:30,20190601,A,2019-06-01 12:31:01,2019-06-01 12:31:01,A02N,Inwood - 207 St
343,042250_A..N,07:02:30,20190601,A,2019-06-01 12:24:21,2019-06-01 12:24:21,A02N,Inwood - 207 St
344,043200_A..N,07:12:00,20190601,A,2019-06-01 12:54:31,2019-06-01 12:54:31,A02N,Inwood - 207 St
345,044250_A..N,07:22:30,20190601,A,2019-06-01 12:40:05,2019-06-01 12:40:05,A02N,Inwood - 207 St
346,044750_A..N,07:27:30,20190601,A,2019-06-01 13:11:30,2019-06-01 13:11:30,A02N,Inwood - 207 St
347,046100_A..N,07:41:00,20190601,A,2019-06-01 12:58:30,2019-06-01 12:58:30,A02N,Inwood - 207 St


In [268]:
df[df.trip_id == '044950_A..S'].sort_values('arrival')

,trip_id,start_time,start_date,route_id,arrival,departure,stop_id,stop_name
1111,044950_A..S,07:29:30,20190601,A,2019-06-01 11:29:30,2019-06-01 11:29:30,A02S,Inwood - 207 St
1117,044950_A..S,07:29:30,20190601,A,2019-06-01 11:31:00,2019-06-01 11:31:00,A03S,Dyckman St
1123,044950_A..S,07:29:30,20190601,A,2019-06-01 11:33:00,2019-06-01 11:33:00,A05S,190 St
1129,044950_A..S,07:29:30,20190601,A,2019-06-01 11:35:00,2019-06-01 11:35:00,A06S,181 St
1135,044950_A..S,07:29:30,20190601,A,2019-06-01 11:36:00,2019-06-01 11:36:00,A07S,175 St
1053,044950_A..S,07:29:30,20190601,A,2019-06-01 11:38:00,2019-06-01 11:38:00,A09S,168 St
1066,044950_A..S,07:29:30,20190601,A,2019-06-01 11:41:30,2019-06-01 11:41:30,A12S,145 St
1079,044950_A..S,07:29:30,20190601,A,2019-06-01 11:45:00,2019-06-01 11:45:00,A15S,125 St
1014,044950_A..S,07:29:30,20190601,A,2019-06-01 11:53:00,2019-06-01 11:53:00,A24S,59 St - Columbus Circle
1030,044950_A..S,07:29:30,20190601,A,2019-06-01 11:55:30,2019-06-01 11:55:30,A27S,42 St - Port Authority Bus Terminal


In [212]:
df = pd.DataFrame({'A':[1,2,3], 'B':['1,2,3,9', '4,5,6', '7,8,9']})

idx = df.index.repeat(df.B.str.split(',').str.len())
df1 = pd.DataFrame({'B': np.concatenate(df.B.str.split(','))}, index = idx)


df1.join(df.drop(['B'], axis=1), how='left')


,B,A
0,1,1
0,2,1
0,3,1
0,9,1
1,4,2
1,5,2
1,6,2
2,7,3
2,8,3
2,9,3


In [76]:
np.nan

nan

In [213]:
df.B.str.split(',')

0    [1, 2, 3, 9]
1       [4, 5, 6]
2       [7, 8, 9]
Name: B, dtype: object

In [75]:
message_collection = [{'vehicle_update': None, 'timestamp': 1559384851, 'trip_update': {'id': '000017E', 'trip_update': {'trip': {'trip_id': '038900_E..S', 'start_date': '20190601', 'route_id': 'E'}, 'stop_time_update': [{'stop_id': 'G05S', 'arrival': 1559384940, 'departure': 1559384940}, {'stop_id': 'G06S', 'arrival': 1559385030, 'departure': 1559385030}, {'stop_id': 'G07S', 'arrival': 1559385180, 'departure': 1559385180}, {'stop_id': 'F05S', 'arrival': 1559385270, 'departure': 1559385270}, {'stop_id': 'F06S', 'arrival': 1559385390, 'departure': 1559385390}, {'stop_id': 'F07S', 'arrival': 1559385480, 'departure': 1559385480}, {'stop_id': 'G08S', 'arrival': 1559385600, 'departure': 1559385600}, {'stop_id': 'G14S', 'arrival': 1559386020, 'departure': 1559386020}, {'stop_id': 'G21S', 'arrival': 1559386380, 'departure': 1559386380}, {'stop_id': 'F09S', 'arrival': 1559386470, 'departure': 1559386470}, {'stop_id': 'F11S', 'arrival': 1559386650, 'departure': 1559386650}, {'stop_id': 'F12S', 'arrival': 1559386740, 'departure': 1559386740}, {'stop_id': 'D14S', 'arrival': 1559386860, 'departure': 1559386860}, {'stop_id': 'A25S', 'arrival': 1559387265, 'departure': 1559387265}, {'stop_id': 'A27S', 'arrival': 1559387700, 'departure': 1559387700}, {'stop_id': 'A28S', 'arrival': 1559387760, 'departure': 1559387760}, {'stop_id': 'A30S', 'arrival': 1559387880, 'departure': 1559387880}, {'stop_id': 'A31S', 'arrival': 1559387970, 'departure': 1559387970}, {'stop_id': 'A32S', 'arrival': 1559388090, 'departure': 1559388090}, {'stop_id': 'A33S', 'arrival': 1559388180, 'departure': 1559388180}, {'stop_id': 'A34S', 'arrival': 1559388270, 'departure': 1559388270}, {'stop_id': 'E01S', 'arrival': 1559388390, 'departure': 1559388390}]}, 'type': 'trip_update'}}]
#message_collection[0]['trip_update']['trip_update']['stop_time_update']
message_collection   

[{'vehicle_update': None,
  'timestamp': 1559384851,
  'trip_update': {'id': '000017E',
   'trip_update': {'trip': {'trip_id': '038900_E..S',
     'start_date': '20190601',
     'route_id': 'E'},
    'stop_time_update': [{'stop_id': 'G05S',
      'arrival': 1559384940,
      'departure': 1559384940},
     {'stop_id': 'G06S', 'arrival': 1559385030, 'departure': 1559385030},
     {'stop_id': 'G07S', 'arrival': 1559385180, 'departure': 1559385180},
     {'stop_id': 'F05S', 'arrival': 1559385270, 'departure': 1559385270},
     {'stop_id': 'F06S', 'arrival': 1559385390, 'departure': 1559385390},
     {'stop_id': 'F07S', 'arrival': 1559385480, 'departure': 1559385480},
     {'stop_id': 'G08S', 'arrival': 1559385600, 'departure': 1559385600},
     {'stop_id': 'G14S', 'arrival': 1559386020, 'departure': 1559386020},
     {'stop_id': 'G21S', 'arrival': 1559386380, 'departure': 1559386380},
     {'stop_id': 'F09S', 'arrival': 1559386470, 'departure': 1559386470},
     {'stop_id': 'F11S', 'arriva